# Portfolio Optimization in Practice

## Key Points

If one could reliably predict future returns, one would not need or seek diversification; one would instead invest in the highest returning asset at each point in time. Crystal balls, however, hardly exist in the real world, and this is the reason why investors should instead hold well-diversified portfolios. Well-diversified portfolios are scientifically defined as portfolios that deliver the highest reward for a given risk budget. In the absence of parameter uncertainty, and if volatility is used as a risk measure, this leaves us with a simple prescription, namely maximize the Sharpe ratio.

This definition, however, is not fully operational because of the presence of parameter uncertainty. In this context, one can turn to naive diversification, which focuses on the question of how to construct a well-balanced portfolio (mean goal) as a proxy for constructing a well-diversified portfolio (end goal). While naive diversification is often taken to imply an equally-weighted portfolio, the approach can be extended to imply an equal allocation in terms of contributions from underlying factors to the risk of the portfolio. In this module, we explore practical issues in the implementation well-diversified portfolios and we work through the mechanics of various portfolio optimization methodologies and discuss their pros and cons in applications to the construction of improved equity benchmarks.

- ### Mean Goal: Well-Balanced Portfolio
- ### End Goal: Wwell-Rewarded Portfolio

## Effective Number of Constituents of a Portfolio
A | B
- | -
<img src="images/effective number of 30 stocks.PNG" width="500"> | <img src="images/effective number of constituents.PNG" width="500">
<img src="images/SPY ENC.PNG" width="500"> | <img src="images/ENC Summary.PNG" width="700">

## Scientific Diversification: MSR

The MSR portoflio requires access to reliable expected return estimates
The GMV portfolio is an efficient portfolio that requires no expected return estimate

Efficient Frontier | Global minimum variance
- | -
<img src="images/efficient frontier.PNG" width="500"> | <img src="images/GMV.PNG" width="500">

Problems with GMV portfolios | Improving GMV
- | -
<img src="images/problems with GMV portfolios.PNG" width="600"> | <img src="images/improving GMV.PNG" width="500">

## Summary
<img src="images/GMV summary.PNG" width="500">


## Measuring Risk Contributions

A | B
- | -
<img src="images/ENC short coming 1.PNG" width="500"> | <img src="images/ENC short coming 2.PNG" width="500">
<img src="images/ENC short coming 3.PNG" width="500"> | <img src="images/ENC short coming 4.PNG" width="500">
<img src="images/ENC short coming 5.PNG" width="500"> | <img src="images/ENC short coming 6.PNG" width="500">
<img src="images/ENC short coming 7.PNG" width="500">

- The calculated value of risk contribution changes when we use correlations instead of assuming they are zero.
- Therefore it is important to include correlation parameters in calculation to give more accurate answer.

# Risk Parity Portfolio

## Simplified Risk Parity Portfolio

In [13]:
v1 = 0.6**2 * 0.151**2
v2 = 0.4**2 * 0.046**2
v12 = 2 * 0.6*0.4 * 0.151*0.046*0.2
v = v1 + v2 + v12
w1 = (v1 + v12)/v
w2 = (v2 + v12)/v

print(w1, w2)

1/(w1**2 + w2**2)

0.9632548620885165 0.10911708345018784


1.0640941498361778

In [15]:
v1 = 0.6**2 * 0.151**2
v2 = 0.4**2 * 0.046**2
v12 = 2 * 0.6*0.4 * 0.151*0.046*0.2
v = v1 + v2 + v12
w1 = (v1 + v12/2)/v
w2 = (v2 + v12/2)/v

print(w1, w2)

1/(w1**2 + w2**2)

0.9270688893191643 0.07293111068083566


1.1563692548941664

- Risk Parity Portfolio is a portfolio with equal risk contribution from contributing assets.
- It maximizes the ENC applied to the risk contributions, as opposed to the dollar contributions.
- Therefore Risk Parity Portfolios are well-balanced interms of risk contributions.
- This quantity is sometimes called effective number of correlated bets (ENCB).
- Risk Partity Portfolio, also known as Equal Risk Contribution Portfolio (ERC), is an inverse vol weighted portofolio if all pairwise correlations are equal.

A | B
- | -
<img src="images/risk parity equation.PNG" width="500"> | <img src="images/risk parity equation 2.PNG" width="500">

- Risk Partity Portfolios are often leveraged to generate same volatility as the benchmark.
- RPP tends to overweight less risky components.
- RPP are extremely popular in practice, because they tends to be more risk efficient than traditional market benchmarks.
- Max ENC = EW, Max ENCB = ERC, Both cases are examples of Naive Diversification.

## Conclusion:
- Cap weighted benchmarks are not the most efficient portfolios because of their excessive concentration
- One can use a variety of diversification methodologies to build more efficient portfolios, including Equally-Weighted, Minimum Variance and Risk Parity Portoflios.

<img src="images/RPP 1.PNG" width="500">
<img src="images/RPP 2.PNG" width="500">
<img src="images/RPP 3.PNG" width="500">
<img src="images/RPP 4.PNG" width="500">
<img src="images/RPP 5.PNG" width="500">
<img src="images/RPP 6.PNG" width="500">


# Risk Contributions and Risk Parity

> The word 'risk' derives from the early Italian risicare, which means 'to dare'. In this sense, risk is a choice rather than a fate. The actions we dare to take, which depend on how free we are to make choices, are what the story of risk is all about.

_Peter L. Bernstein, Against the Gods: The Remarkable Story of Risk_

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize

import edhec_risk_kit as erk

#%load_ext autoreload
#%autoreload 2

inds = ['Food', 'Beer', 'Smoke', 'Games', 'Books', 'Hshld', 'Clths', 'Hlth',
       'Chems', 'Txtls', 'Cnstr', 'Steel', 'FabPr', 'ElcEq', 'Autos', 'Carry',
       'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'BusEq', 'Paper',
       'Trans', 'Whlsl', 'Rtail', 'Meals', 'Fin', 'Other']
inds=['Beer', 'Hlth', 'Fin','Rtail','Whlsl']
ind_rets = erk.get_ind_returns(weighting="vw", n_inds=49)["1974":]
ind_mcap = erk.get_ind_market_caps(49, weights=True)["1974":]

rets = ind_rets["2013":][inds]
cov = rets.cov()

In [4]:
def risk_contribution(w, cov):
    """
    Returns the weights of the portfolio that equalizes the contributions
    of the constituents based on the given covariance matrix
    """
    # Warning, pandas does have built-in outer product so w.T@w does not work
    var = (w*cov).multiply(w,axis=0)
    risk_contribution = var.sum(axis=1) / var.sum().sum()
    return risk_contribution

In [5]:
risk_contribution(erk.weight_ew(rets), cov).plot.bar(title="Risk Contributions of an EW portfolio", 
                                                     color=['r','b','g','k','m'])
plt.show()

<Figure size 432x288 with 1 Axes>

In [7]:
def target_risk_contributions(target_risk, cov):
    """
    Returns the weights of the portfolio that gives you the weights such
    that the contributions to portfolio risk are as close as possible to
    the target_risk, given the covariance matrix
    """
    def cost(w, target_risk, cov):
        w_contribs = risk_contribution(w, cov)
        return ((target_risk - w_contribs)**2).sum()
    
    init_w = np.repeat(1/len(cov), len(cov))
    
    test_c = cost(init_w, target_risk, cov)
    
    bounds = ((0.0, 1.0),) * len(cov)
    weights_sum_to_1 = {'type': 'eq',
                        'fun': lambda weights: np.sum(weights) - 1}
    
    result = minimize(cost,
             init_w,
             args=(target_risk, cov),
             method='SLSQP',
             bounds=bounds,
             constraints=weights_sum_to_1
            )
    return result.x

def equal_risk_contributions(cov):
    """
    Returns the weights of the portfolio that equalizes the contributions
    of the constituents based on the given covariance matrix
    """
    equal_risk = np.repeat(1/len(cov), len(cov))
    weight = target_risk_contributions(equal_risk, cov)
    return weight

In [8]:
risk_contribution(equal_risk_contributions(cov), cov).plot.bar(title="Risk Contributions of an ERC portfolio")
plt.show()

<Figure size 432x288 with 1 Axes>

In [ ]:
## Back testing the Equal Risk Contribution Portfolio

In [36]:
def weight_erc(r, cov_estimator=erk.sample_cov, **kwargs):
    """
    Produces the weights of the ERC portfolio given a covariance matrix of the returns 
    """
    est_cov = cov_estimator(r, **kwargs)
    return equal_risk_contributions(est_cov)

In [ ]:
ewr = erk.backtest_ws(ind_rets, estimation_window=36, weighting=erk.weight_ew)
cwr = erk.backtest_ws(ind_rets, estimation_window=36, weighting=erk.weight_cw, cap_weights=ind_mcap)
mv_erc_r = erk.backtest_ws(ind_rets, estimation_window=36, weighting=weight_erc, cov_estimator=erk.sample_cov)
btr = pd.DataFrame({"EW": ewr, "CW": cwr, "ERC-Sample": mv_erc_r})
(1+btr).cumprod().plot(figsize=(12,6), title="Industry Portfolios")
erk.summary_stats(btr.dropna())

# Week 4 Quiz

In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize

import edhec_risk_kit as erk

#%load_ext autoreload
#%autoreload 2

In [ ]:
ind_rets = erk.get_ind_returns(weighting="vw", n_inds=49)["2014":]
ind_mcap = erk.get_ind_market_caps(49, weights=True)["2014":]

In [25]:
cov = ind_rets.cov()
w_cw = ind_mcap.iloc[0]
risk_contribution(w_cw, cov).sort_values().tail()

Rtail    0.070120
Drugs    0.084653
Oil      0.088914
Softw    0.093745
Banks    0.104060
dtype: float64

In [41]:
cov = ind_rets.cov()
w_ew = erk.weight_ew(ind_rets)
risk_contribution(w_ew, cov).sort_values().tail()

BldMt    0.028106
Mines    0.028641
Coal     0.029025
Ships    0.030005
Steel    0.030938
dtype: float64

In [42]:
w_er = pd.Series(weight_erc(ind_rets, cov_estimator=erk.sample_cov), index=cov.index)

In [43]:
w_er.sort_values().tail()

Food     0.030251
Beer     0.032028
Smoke    0.032606
Hshld    0.033184
Util     0.052156
dtype: float64

In [35]:
w_er.sort_values().head()

Steel    0.012809
Ships    0.013041
BldMt    0.013790
Mines    0.013951
Mach     0.014292
dtype: float64

In [48]:
r_contribs = risk_contribution(w_cw, cov).sort_values()
r_contribs[-1]-r_contribs[0]

0.10396449439816839

In [49]:
r_contribs = risk_contribution(w_ew, cov).sort_values()
r_contribs[-1]-r_contribs[0]

0.025021757624911472